In [1]:
import pandas as pd
import random
from tqdm import tqdm
import gensim
from gensim.models import Word2Vec 
from gensim.parsing.preprocessing import preprocess_documents
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

# References:
- 


In [3]:
root = './RASA/data/'
model_root='./RASA/offline_models/'
processed = './Data/processing/Processed_Airbnb/'
raw = './Data/raw/'

def create_embeddings(text_corpus_listing, name):
    processed_text_corpus = preprocess_documents(text_corpus_listing)
    tagged_text_corpus = [TaggedDocument(d, [i]) for i, d in enumerate(processed_text_corpus)]
    text_corpus_model = Doc2Vec(tagged_text_corpus, dm=0, vector_size=200, window=2, min_count=1, epochs=100, hs=1)

    text_corpus_model.save(root+'embeddings/'+ name +'_embeddings')

    return text_corpus_model

In [3]:
listings = pd.read_csv(raw+'listings.csv.gz', sep=',')
reviews = pd.read_csv(processed+'ratings_filter.csv', sep=',')
listings=listings.rename(columns={"id": "listing_id"})
listings.to_csv(processed+'listings.csv.gz', sep=',')

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


### User Review Embeddings

In [8]:
reviews = reviews.dropna()
print("No.of.reviews:"+str(len(reviews)))
users = reviews["reviewer_id"].unique().tolist()
print("No.of.users:"+str(len(users)))

No.of.reviews:264360
No.of.users:257032


In [9]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 264360 entries, 0 to 272484
Data columns (total 8 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Unnamed: 0     264360 non-null  object 
 1   listing_id     264360 non-null  float64
 2   id             264360 non-null  float64
 3   date           264360 non-null  object 
 4   reviewer_id    264360 non-null  float64
 5   reviewer_name  264360 non-null  object 
 6   comments       264360 non-null  object 
 7   rating         264360 non-null  float64
dtypes: float64(4), object(4)
memory usage: 18.2+ MB


In [86]:
comment_corpus_model = create_embeddings(reviews['comments'].values,'review')

In [107]:
new_doc = gensim.parsing.preprocessing.preprocess_string("private room dishwasher")
test_doc_vector = comment_corpus_model.infer_vector(new_doc)
sims = comment_corpus_model.docvecs.most_similar(positive = [test_doc_vector])
topK = 5
for s in sims[:topK]:
    print(f"{(s[1])} | {reviews['listing_id'].iloc[s[0]]}")

<ipython-input-107-d011b8cf41a5>:3: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  sims = comment_corpus_model.docvecs.most_similar(positive = [test_doc_vector])


0.6630650162696838 | 23626417.0
0.6610732674598694 | 31945057.0
0.6557047963142395 | 35632344.0
0.6534749269485474 | 37927536.0
0.6384268403053284 | 35855352.0


### Listing word embeddings

In [23]:
listings = pd.read_csv(raw+'listings.csv.gz', sep=',')
print("No.of.listings:"+str(len(listings)))
listings.info()

No.of.listings:5402
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5402 entries, 0 to 5401
Data columns (total 74 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            5402 non-null   int64  
 1   listing_url                                   5402 non-null   object 
 2   scrape_id                                     5402 non-null   int64  
 3   last_scraped                                  5402 non-null   object 
 4   name                                          5402 non-null   object 
 5   description                                   5392 non-null   object 
 6   neighborhood_overview                         3908 non-null   object 
 7   picture_url                                   5402 non-null   object 
 8   host_id                                       5402 non-null   int64  
 9   host_url                                   

In [10]:
# choose columns to generate embeddings.
listings = listings[['id','listing_url','name','description','neighborhood_overview','picture_url', 
'property_type','room_type','accommodates','bathrooms','bathrooms_text',                               
'bedrooms','beds','amenities','price','minimum_nights','maximum_nights','review_scores_rating',                         
'review_scores_accuracy','review_scores_cleanliness','review_scores_checkin',
'review_scores_communication','review_scores_location']]

listings.fillna('0', inplace=True)

listings.reset_index(drop = True, inplace = True)

In [11]:
import string
def remove_punc(sample_str):
    # Create translation table in which special charcters
    # are mapped to empty string
    translation_table = str.maketrans('', '', string.punctuation)
    # Remove special characters from the string using translation table
    sample_str = sample_str.translate(translation_table)
    return sample_str

In [12]:
listings['words_features'] = listings['amenities'].apply(remove_punc)

for ind in listings.index:
     listings['review_scores_rating'][ind] = (float(listings['review_scores_rating'][ind]) + float(listings['review_scores_accuracy'][ind]) + float(listings['review_scores_cleanliness'][ind]) + float(listings['review_scores_checkin'][ind]) + float(listings['review_scores_communication'][ind]) + float(listings['review_scores_location'][ind]))
     listings['review_scores_rating'][ind] = (listings['review_scores_rating'][ind])/6
     listings['words_features'][ind] = 'amenities:'+listings['words_features'][ind] +'description:'+  listings['description'][ind] +'neighborhood_overview:'+  listings['neighborhood_overview'][ind]+'property_type:'+  listings['property_type'][ind]+'room_type:'+  listings['room_type'][ind]+'accommodates:'+  str(listings['accommodates'][ind])+'bedrooms:'+  str(listings['bedrooms'][ind])+'beds:'+  str(listings['beds'][ind])+'price range:'+  listings['price'][ind]
listings = listings.rename(columns={"review_scores_rating": "overall_rating"})
listings["overall_rating"].fillna(0, inplace=True)

<ipython-input-12-8769852d1023>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  listings['review_scores_rating'][ind] = (float(listings['review_scores_rating'][ind]) + float(listings['review_scores_accuracy'][ind]) + float(listings['review_scores_cleanliness'][ind]) + float(listings['review_scores_checkin'][ind]) + float(listings['review_scores_communication'][ind]) + float(listings['review_scores_location'][ind]))
<ipython-input-12-8769852d1023>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  listings['review_scores_rating'][ind] = (listings['review_scores_rating'][ind])/6
<ipython-input-12-8769852d1023>:6: SettingW

In [13]:
text_corpus_model = create_embeddings(listings['words_features'].values,'list')

new_doc = gensim.parsing.preprocessing.preprocess_string("private room dishwasher")
test_doc_vector = text_corpus_model.infer_vector(new_doc)
sims = text_corpus_model.docvecs.most_similar(positive = [test_doc_vector])
for s in sims:
    print(f"{(s[1])} | {listings['listing_url'].iloc[s[0]]}")

0.4589652717113495 | https://www.airbnb.com/rooms/25991654
0.4400947391986847 | https://www.airbnb.com/rooms/40247038
0.4249565899372101 | https://www.airbnb.com/rooms/8641456
0.4234083890914917 | https://www.airbnb.com/rooms/51961840
0.41909921169281006 | https://www.airbnb.com/rooms/49927889
0.4160788655281067 | https://www.airbnb.com/rooms/16685383
0.41239070892333984 | https://www.airbnb.com/rooms/32729549
0.4093344509601593 | https://www.airbnb.com/rooms/39642322
0.4028607904911041 | https://www.airbnb.com/rooms/28220320
0.40280336141586304 | https://www.airbnb.com/rooms/21593642


<ipython-input-13-75f26069de46>:5: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  sims = text_corpus_model.docvecs.most_similar(positive = [test_doc_vector])


### Content-based filtering
- https://github.com/SarangDeshmukh7/Recommendation-Engine/blob/master/Content_Based_Filtering.ipynb

In [108]:
users = reviews["reviewer_id"].unique().tolist()
random.shuffle(users)

In [109]:
reviews_listing = pd.merge(listings,reviews)

In [7]:

reviews_listing['listing_id']= reviews_listing['listing_id'].astype(str)

# extract 90% of user ID's
users_train = [users[i] for i in range(round(0.9*len(users)))]
#split data into train and validation set
train_df = reviews_listing[reviews_listing['reviewer_id'].isin(users_train)]
validation_df = reviews_listing[~reviews_listing['reviewer_id'].isin(users_train)]

In [10]:
#list to capture watch history of the users
watch_train = []

# populate the list with the movie ID
for i in tqdm(users_train):
    temp = train_df[train_df["reviewer_id"] == i]["listing_id"].tolist()
    watch_train.append(temp)

100%|██████████| 233485/233485 [03:16<00:00, 1189.39it/s]


In [13]:
# train word2vec model
model = Word2Vec(window = 10, sg = 1, hs = 0,
                 negative = 10, 
                 alpha=0.03, min_alpha=0.0007,
                 seed = 14)

model.build_vocab(watch_train, progress_per=200)

model.train(watch_train, total_examples = model.corpus_count, 
            epochs=10, report_delay=1)


(2375641, 2401840)

In [14]:

model.train(watch_train, total_examples = model.corpus_count, 
            epochs=10, report_delay=1)
model.save(model_root+'ContentBasedFilter')

In [56]:
watch = train_df[["listing_id","listing_url","name"]]

# remove duplicates
watch.drop_duplicates(inplace=True, subset='listing_id', keep="last")

# create movie id and tittle dictionary
watch_dict = watch.groupby('listing_id')['listing_id'].apply(list).to_dict()

def similar_watch(v, n = 5):

    # extract most similar movies for the input vector
    ms = model.wv.similar_by_vector(v, topn= n+1)[1:]
    # extract name and similarity score of the similar movies
    new_ms = []
    for j in ms:
        pair = (watch_dict[j[0]][0], j[1])
        new_ms.append(pair)
        
    return new_ms        

<ipython-input-56-b3d13b8f8448>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  watch.drop_duplicates(inplace=True, subset='listing_id', keep="last")


In [54]:
print(watch_dict['2818'])

['Quiet Garden View Room & Super Fast WiFi']


In [57]:
similar_watch('2818',5)

[('1383369', 0.9963660836219788),
 ('505650', 0.9962319135665894),
 ('18882385', 0.9962259531021118),
 ('28170875', 0.9961230158805847),
 ('23332938', 0.9960837364196777)]

### Collaborative filtering

- Refer notebook ColaborativeFiltering.ipynb

In [ ]:
!pip3 install pyspark

# importing all the libraries we’ll require to build the book recommender
import sys
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions  import *
from pyspark.sql.types import *
from pyspark.ml.recommendation import ALS,ALSModel

# define the configurations for this Spark program
conf = SparkConf().setMaster("local[*]").setAppName("airbnb")
conf.set("spark.executor.memory", "6G")
conf.set("spark.driver.memory", "2G")
conf.set("spark.executor.cores", "4")
conf.set("spark.serializer", "org.apache.spark.serializer.KryoSerializer")
conf.set("spark.default.parallelism", "4")

# create a Spark Session instead of a Spark Context
spark = SparkSession.builder \
    .config(conf = conf) \
  .appName("spark session example") \
  .getOrCreate()

listings_df = spark.read.option("delimiter", ",").option("header", "true").csv('/content/drive/MyDrive/299/listings.csv')
listings_df.show(2)

listings_df.select('id').distinct().show(2)

user_ratings_df = spark.read.option("delimiter", ",").option("header", "true").csv('/content/drive/MyDrive/299/ratings_filter.csv')
# Columns User-ID, ISBN and Book-Rating were in string format, which we convert to int
ratings_df = user_ratings_df.withColumn("reviewer_id",
                                        user_ratings_df['reviewer_id'].\
                                        cast(IntegerType())).\
										withColumn("listing_id", user_ratings_df['listing_id'].\
           								cast(IntegerType())).\
    									withColumn("rating",\
                                        user_ratings_df['rating'].\
                                  		cast(IntegerType())).\
        								na.drop()
ratings_df.show(2)

# define parameters
als = ALS(maxIter=5, regParam=0.01, userCol="reviewer_id", itemCol="listing_id", ratingCol="rating",coldStartStrategy="drop")
#fit the model to the ratings
model = als.fit(ratings_df)

model.save("/content/drive/MyDrive/299/als_model")

user_ratings_df = spark.read.option("delimiter", ",").option("header", "true").csv('/content/drive/MyDrive/299/ratings_filter.csv')
# Columns User-ID, ISBN and Book-Rating were in string format, which we convert to int
ratings_df = user_ratings_df.withColumn("reviewer_id",
                                        user_ratings_df['reviewer_id'].\
                                        cast(IntegerType())).\
										withColumn("listing_id", user_ratings_df['listing_id'].\
           								cast(IntegerType())).\
    									withColumn("rating",\
                                        user_ratings_df['rating'].\
                                  		cast(IntegerType())).\
        								na.drop()
ratings_df.show(2)

### Help link Embeddings

In [1]:
import json
from typing import Any, Text, Dict, List
import torch
from bert_serving.client import BertClient
from rasa_sdk import Action, Tracker
from rasa_sdk.executor import CollectingDispatcher
import numpy as np
from sentence_transformers import SentenceTransformer

# sentence embedding selection
sentence_transformer_select=True
pretrained_model='stsb-roberta-large' # Refer: https://github.com/UKPLab/sentence-transformers/blob/master/docs/pretrained-models/nli-models.md
score_threshold = 0.70  # This confidence scores can be adjusted based on your need!!

In [4]:

def encode_standard_question(sentence_transformer_select=True, pretrained_model='bert-base-nli-mean-tokens'):
    """
    This will encode all the questions available in question database into sentence embedding. The result will be stored into numpy array for comparision purpose.
    """
    if sentence_transformer_select:
        bc = SentenceTransformer(pretrained_model)
    else:
        bc = BertClient(check_version=False)
    data = json.load(open(raw+"/faq.json", "rt", encoding="utf-8"))
    standard_questions = [each['q'].replace('-',' ') for each in data]
    print("Standard question size", len(standard_questions))
    print("Start to calculate encoder....")
    if sentence_transformer_select:
        standard_questions_encoder = torch.tensor(bc.encode(standard_questions)).numpy()
    else:
        standard_questions_encoder = bc.encode(standard_questions)
    np.save(root+"embeddings/questions_embedding", standard_questions_encoder)
    standard_questions_encoder_len = np.sqrt(np.sum(standard_questions_encoder * standard_questions_encoder, axis=1))
    np.save(root+"embeddings/questions_embedding_len", standard_questions_encoder_len)


encode_standard_question(sentence_transformer_select,pretrained_model)

Standard question size 86
Start to calculate encoder....
